In [18]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.utils.data as td
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from matplotlib import image
from matplotlib import pyplot
import time

import torchvision
from torchvision.transforms.functional import normalize
from torchvision.transforms.transforms import ToTensor
from torchvision.transforms.transforms import Resize
from torchvision.transforms.transforms import RandomHorizontalFlip
from torchvision.transforms.transforms import Normalize


PATH_TO_DATASET="/content/drive/MyDrive/chest_xray/train"


In [19]:
def load_data(path, test_split, val_split, batch_size, input_size):
    
    normalize = transforms.Normalize(
        mean=[0.4914, 0.4822, 0.4465],
        std=[0.2023, 0.1994, 0.2010],
    )
    ######## Write your code here ########
    transform_dict={"src":transforms.Compose([
      transforms.Resize((227,227)),
            transforms.ToTensor(),
            normalize])}
      
    data=torchvision.datasets.ImageFolder(root=PATH_TO_DATASET,transform=transform_dict["src"])
#dataset = TensorDataset(x_tensor, y_tensor)
#val_size = int(len(dataset)*0.2)
#train_size = len(dataset)- int(len(dataset)*0.2)
#train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
    train_split=1-test_split-val_split
    train_size=int(len(data)*train_split)
    val_size=int(len(data)*val_split)
    test_size=int(len(data)*test_split)
    print("The train size, val size and test size is resp ",train_size," ",val_size," ",test_size," ",len(data))
    train,val,test=torch.utils.data.random_split(data,[train_size,val_size,test_size])
    data_loader_train = torch.utils.data.DataLoader(train, batch_size=batch_size,
                                          shuffle=True, num_workers=0, drop_last=False)
    data_loader_test = torch.utils.data.DataLoader(test, batch_size=batch_size,
                                          shuffle=True, num_workers=0, drop_last=False)
    data_loader_val = torch.utils.data.DataLoader(val, batch_size=batch_size,
                                          shuffle=True, num_workers=0, drop_last=False)
    return data_loader_train, data_loader_test, data_loader_val


In [20]:
data_loader_train, data_loader_test, data_loader_val=load_data(PATH_TO_DATASET,0.25,0.25,32,40)

The train size, val size and test size is resp  2928   1464   1464   5856


In [21]:
class VGG16(nn.Module):
    def __init__(self, num_classes=10):
        super(VGG16, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU())
        self.layer2 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(), 
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU())
        self.layer4 = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer5 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU())
        self.layer6 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU())
        self.layer7 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer8 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())
        self.layer9 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())
        self.layer10 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer11 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())
        self.layer12 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())
        self.layer13 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(7*7*512, 4096),
            nn.ReLU())
        self.fc1 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU())
        self.fc2= nn.Sequential(
            nn.Linear(4096, num_classes))
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.layer6(out)
        out = self.layer7(out)
        out = self.layer8(out)
        out = self.layer9(out)
        out = self.layer10(out)
        out = self.layer11(out)
        out = self.layer12(out)
        out = self.layer13(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [22]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

num_classes = 3
num_epochs = 20
batch_size = 16
learning_rate = 0.005

model = VGG16(num_classes).to(device)


# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.005, momentum = 0.9)  


# Train the model
total_step = len(data_loader_train)

In [23]:
total_step = len(data_loader_train)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(data_loader_train):  
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
            
    # Validation
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in data_loader_train:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            del images, labels, outputs
    
        print('Accuracy of the network on the {} validation images: {} %'.format(5000, 100 * correct / total)) 

Epoch [1/20], Step [92/92], Loss: 0.4475
Accuracy of the network on the 5000 validation images: 74.21448087431693 %
Epoch [2/20], Step [92/92], Loss: 1.0172
Accuracy of the network on the 5000 validation images: 70.86748633879782 %
Epoch [3/20], Step [92/92], Loss: 0.5944
Accuracy of the network on the 5000 validation images: 75.03415300546447 %
Epoch [4/20], Step [92/92], Loss: 0.5565
Accuracy of the network on the 5000 validation images: 82.00136612021858 %
Epoch [5/20], Step [92/92], Loss: 0.4853
Accuracy of the network on the 5000 validation images: 81.4207650273224 %
Epoch [6/20], Step [92/92], Loss: 0.3276
Accuracy of the network on the 5000 validation images: 80.94262295081967 %
Epoch [7/20], Step [92/92], Loss: 0.6611
Accuracy of the network on the 5000 validation images: 80.9084699453552 %
Epoch [8/20], Step [92/92], Loss: 0.5165
Accuracy of the network on the 5000 validation images: 82.71857923497268 %
Epoch [9/20], Step [92/92], Loss: 0.9424
Accuracy of the network on the 50

In [24]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in data_loader_test:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        del images, labels, outputs

    print('Accuracy of the network on the {} test images: {} %'.format(10000, 100 * correct / total))   

Accuracy of the network on the 10000 test images: 80.39617486338798 %
